In [ ]:
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers
from sklearn.model_selection import train_test_split

In [ ]:
def load_data(path):
    return (
        pd.read_csv(path)
        .pivot(index="frame", columns="track_id", values=["x", "y"])
        .interpolate(method="linear", limit_area="inside", limit_direction="both")
    )

In [ ]:
def split_sequence(df, n_steps = 30, n_steps_forward = 30):
    X, y = [], []
    for track_id in df.columns.levels[1]:
        track = df.xs(track_id, level=1, axis=1).dropna()
        if len(track) >= n_steps + n_steps_forward:
            for i in range(len(track) - n_steps - n_steps_forward):
                X.append([*track.iloc[i : i + n_steps].to_numpy()])
                y.append([*track.iloc[i + n_steps: i + n_steps + n_steps_forward].to_numpy()])

    return np.array(X), np.array(y)

In [ ]:
df = pd.read_csv('sequence.csv')
grouped_data = df.groupby('track_id')

for track_id, group in grouped_data:
    output_name = f"sequence_{track_id}.csv"
    group.to_csv(output_name, index=False)

In [ ]:
def calculate_ade_fde(y_true, y_pred):
    ade = np.mean(np.sqrt(np.sum(np.square(y_true - y_pred), axis=-1)))
    fde = np.sqrt(np.sum(np.square(y_true[-1, :] - y_pred[-1, :])))
    return ade, fde

In [ ]:
n_steps = 15
n_steps_forward = 15
n_features = 2
num_trials = 5
all_ade = []
all_fde = []
file_name = "sequence_1.csv"
df = load_data(file_name)

In [ ]:
X, y = split_sequence(df, n_steps, n_steps_forward)
X = X.reshape((X.shape[0], X.shape[1], n_features))
y = y.reshape((y.shape[0], n_steps_forward, n_features))
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, shuffle=False)

In [ ]:
for trial in range(num_trials):
    
    model = tf.keras.Sequential()
    model.add(tf.keras.layers.LSTM(128, activation="relu", input_shape=(n_steps, n_features), return_sequences=True))
    model.add(tf.keras.layers.Dense(2))
    model.compile(optimizer=tf.keras.optimizers.Adam(0.003), loss=tf.keras.losses.MeanAbsoluteError())
    
    early_stopping_callback = tf.keras.callbacks.EarlyStopping(
                monitor='val_loss',
                patience=10,
                restore_best_weights=True)
    
    model.fit(X_train, y_train, epochs=300, batch_size=8, verbose=1,
              validation_data=(X_test, y_test), callbacks=[early_stopping_callback])
    predicted_values = model.predict(X_test, verbose=1)
    
    for i in range(len(y_test)):
        print(f"{file_name} - Rzeczywiste (X, Y): {y_test[i]}, Prognozowane (X, Y): {predicted_values[i]}")
        
    trial_ade, trail_fde = 0, 0
    
    for i in range(len(y_test)):
        ade, fde = calculate_ade_fde(y_test[i], predicted_values[i])
        trial_ade += ade
        trial_fde += fde
        
    avg_ade = trial_ade / len(y_test)
    avg_fde = trial_fde / len(y_test)
    all_ade.append(avg_ade)
    all_fde.append(avg_fde)

In [ ]:
final_avg_ade = sum(all_ade) / num_trials
final_avg_fde = sum(all_fde) / num_trials

print(f"Ostateczna Średnia ADE: {final_avg_ade}, Ostateczna Średnia FDE: {final_avg_fde}")